In [1]:
!pip install boto3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.6/10.6 MB 54.2 MB/s eta 0:00:0000:010:01
  Attempting uninstall: botocore
    Found existing installation: botocore 1.27.59
    Uninstalling botocore-1.27.59:
      Successfully uninstalled botocore-1.27.59
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.4.2 requires botocore<1.27.60,>=1.27.59, but you have botocore 1.29.109 which is incompatible.


In [2]:
!cp -r /kaggle/input/datasetuno/metal_nut/* /kaggle/working

In [3]:
import boto3
import uuid, os

ACCESS_KEY = os.environ.get('AWS_ACCESS_KEY_ID')
SECRET_KEY = os.environ.get('AWS_SECRET_ACCESS_KEY')
BUCKET_NAME = os.environ.get('BUCKET_NAME')

DATASET_NAME = "metal_nut"
GENERATOR_NAME = "generator_weights.pt"
GENERATOR_PATH = f"{DATASET_NAME}/{GENERATOR_NAME}"
DESCRIMINATOR_NAME = "descriminator_weights.pt"
DESCRIMINATOR_PATH = f"{DATASET_NAME}/{DESCRIMINATOR_NAME}"
IMAGES_NAME = "images.zip"
session = boto3.Session(
    aws_access_key_id=ACCESS_KEY, 
    aws_secret_access_key=SECRET_KEY
)

s3 = session.resource('s3')
bucket = s3.Bucket(BUCKET_NAME)

## DOWNLOADING THE GENERATOR

In [ ]:
try:
    bucket.download_file(Key=GENERATOR_PATH, Filename=GENERATOR_NAME)
except:
    print("The generator weights don't exist.")

## DOWNLOADING THE DESCRIMINATOR

In [ ]:
try:
    bucket.download_file(Bucket=BUCKET_NAME, Key=DESCRIMINATOR_PATH, Filename=DESCRIMINATOR_NAME)
except:
    print("The descriminator weights don't exist.")

In [ ]:
!rm -rf images/*

## DCGAN Architecture

In [ ]:
!python dcgan.py --n_epochs 10 --batch_size 64 --lr 0.0002 --b1 0.5 --b2 0.999 --n_cpu 8 --latent_dim 100 --img_size 256 --channels 3 --sample_interval 50 --train_data False --save_weights False --augment_data False --load_weights True --save_path "augment" --count 100 --weights_filename "weights.pt"

## INFOGAN Architecture

In [ ]:
#!python infogan.py --n_epochs 5000 --batch_size 64 --lr 0.0002 --b1 0.5 --b2 0.999 --n_cpu 8 --latent_dim 100 --n_classes 1 --img_size 256 --channels 3 --sample_interval 50 --save_weights True --augment_data True --load_weights True --save_path "augment" --count 100 --weights_filename "weights.pt"

## Upload or Overwrite the Generator weights

In [ ]:
try:
    bucket.upload_file(Key=GENERATOR_PATH, Filename=GENERATOR_NAME)
    print(f'File {GENERATOR_NAME} uploaded to S3 bucket {BUCKET_NAME}')
except Exception as e:
    print(f"Something went wrong: {e}")

## Upload or Overwrite the Descriminator weights

In [ ]:
try:
    bucket.upload_file(Key=DESCRIMINATOR_PATH, Filename=DESCRIMINATOR_NAME)
    print(f'File {DESCRIMINATOR_NAME} uploaded to S3 bucket {BUCKET_NAME}')
except:
    print("Something went wrong")

In [ ]:
!zip images.zip images/*

## Upload or Overwrite the Training Images

In [ ]:
IMAGES_PATH = f"{DATASET_NAME}/images-{uuid.uuid4()}.zip"
try:
    bucket.upload_file(Key=IMAGES_PATH, Filename=IMAGES_NAME)
    print(f'File {IMAGES_NAME} uploaded to S3 bucket {BUCKET_NAME}')
except:
    print("Couldn't upload images")

## CREATE GIF

In [21]:
import os
import imageio
import numpy as np
from PIL import Image, ImageDraw

def create_gif(input_dir, output_file, fps=2):
    # Collect image files from the input directory
    image_files = [os.path.join(input_dir, f) for f in os.listdir(input_dir) 
                   if os.path.isfile(os.path.join(input_dir, f)) and f.endswith('.png')]
    # Sort the files based on their name
    image_files.sort(key=lambda f: int(''.join(filter(str.isdigit, f))))

    # Create a list of NumPy arrays from the image files
    images = []
    for file in image_files:
        with Image.open(file) as img:
            # Add a text caption to the image indicating its file name
            draw = ImageDraw.Draw(img)
            draw.text((0, 0), os.path.basename(file))
            images.append(np.array(img))

    # Save the list of NumPy arrays as an animated GIF
    with imageio.get_writer(output_file, mode='I', fps=fps) as writer:
        for img in images:
            writer.append_data(img)

    print(f'Animated GIF created successfully: {output_file}')


In [23]:
create_gif('images', 'images.gif')

Animated GIF created successfully: images.gif


In [24]:
try:
    bucket.upload_file(Key=f"{DATASET_NAME}/images.gif", Filename='images.gif')
except:
    print("GIF don't exist.")